In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# Data Acquisition

1. Read the case, department, and source data into their own spark dataframes.

In [3]:
# Case df
case = spark.read.csv("case.csv", header=True, inferSchema=True)
case.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [4]:
# Department df
dept = spark.read.csv("dept.csv", header=True, inferSchema=True)

In [5]:
# Source df
source = spark.read.csv("source.csv", header=True, inferSchema=True)

2. Let's see how writing to the local disk works in spark:

* Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
* Inspect your folder structure. What do you notice?

In [6]:
# save locally as csv
case.write.csv("case_csv", mode="overwrite")
dept.write.csv("dept_csv", mode="overwrite")
source.write.csv("source_csv", mode="overwrite")
# save locally as json
case.write.json("case_json", mode="overwrite")
dept.write.json("dept_json", mode="overwrite")
source.write.json("source_json", mode="overwrite")

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [7]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [8]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [9]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [10]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [11]:
# convert string dates to datetime

In [12]:
case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', "M/d/yy H:mm"))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', "M/d/yy H:mm"))\
.withColumn('SLA_due_date', to_timestamp('SLA_due_date', "M/d/yy H:mm"))
case.show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO                   
 num_days_late        | -2.0126041

# Questions

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [87]:
case = (
    case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
    .withColumn("case_age_SLA", datediff(current_timestamp(), "SLA_due_date"))
)

In [88]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1318                 
 late_SLA             | 319                  
 case_age_SLA         | 319                  
only showing top 1 row



In [89]:
# The oldest currently open issue with regards to SLA = 1667 days
(
case
.filter(case.case_closed == 'NO')
.sort(col("case_age_SLA").desc())
.show(1, vertical=True)
    )

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | YES                  
 num_days_late        | 348.6458333          
 case_closed          | NO                   
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age             | 1683                 
 late_SLA             | 1667                 
 case_age_SLA         | 1667                 
only showing top 1 row



In [53]:
# The oldest currently open issue with regards to open = 1683 days
(
case
.filter(case.case_closed == 'NO')
.sort(col("case_age").desc())
.show(1, vertical=True)
    )

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | YES                  
 num_days_late        | 348.6458333          
 case_closed          | NO                   
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age             | 1683                 
only showing top 1 row



2. How many Stray Animal cases are there?

In [54]:
(
case
.filter(case.service_request_type == 'Stray Animal')
.agg(count('service_request_type'))
.sort(col("count(service_request_type)").desc())
.show(3, vertical=True)
    )

-RECORD 0----------------------------
 count(service_request_type) | 26760 



3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [55]:
(
case
.filter(case.dept_division == 'Field Operations')
.groupby('dept_division')
.agg(count('dept_division'))
.show(3, vertical=True)
    )

-RECORD 0--------------------------------
 dept_division        | Field Operations 
 count(dept_division) | 116915           



4. Convert the council_district column to a string column.

In [56]:
case = case.withColumn("council_district", col("council_district").cast("string"))

In [57]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int')]

5. Extract the year from the case_closed_date column.

In [58]:
(
case.withColumn("year", year("case_closed_date")).show(1, vertical=True)
)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1318                 
 year                 | 2018                 
only showing top 1 row



6. Convert num_days_late from days to hours in new columns num_hours_late.

In [59]:
case.withColumn("num_hours_late", expr("num_days_late * 24")).show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1318                 
 num_hours_late       | -23964.2102784       
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05

7. Join the case data with the source and department data.

In [60]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [61]:
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [62]:
merged = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # left join source on source_id
    .join(source, 'source_id', 'left')
)

In [63]:
merged.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1318                 
 department           | Animal Care Services 
 dept_subject_to_SLA  | YES                  
 source_username      | svcCRMLS             
-RECORD 1------------------------------------
 source_id            | svcCRMSS             
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

8. Are there any cases that do not have a request source?

In [66]:
# NO
case.filter("source_id is null").show(vertical=True)

(0 rows)



9. What are the top 10 service request types in terms of number of requests?

In [135]:
(
case
.groupby('service_request_type')
.agg(count('service_request_type'))
.sort(col('count(service_request_type)').desc())
.show(10, vertical=True)
        )

-RECORD 0-------------------------------------------
 service_request_type        | No Pickup            
 count(service_request_type) | 86855                
-RECORD 1-------------------------------------------
 service_request_type        | Overgrown Yard/Trash 
 count(service_request_type) | 65895                
-RECORD 2-------------------------------------------
 service_request_type        | Bandit Signs         
 count(service_request_type) | 32910                
-RECORD 3-------------------------------------------
 service_request_type        | Damaged Cart         
 count(service_request_type) | 30338                
-RECORD 4-------------------------------------------
 service_request_type        | Front Or Side Yar... 
 count(service_request_type) | 28794                
-RECORD 5-------------------------------------------
 service_request_type        | Stray Animal         
 count(service_request_type) | 26760                
-RECORD 6-------------------------------------

In [69]:
case.groupby('service_request_type').count().sort(col('count').desc()).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|86855|
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|        Damaged Cart|30338|
|Front Or Side Yar...|28794|
|        Stray Animal|26760|
|Aggressive Animal...|24882|
|Cart Exchange Req...|22024|
|Junk Vehicle On P...|21473|
|     Pot Hole Repair|20616|
+--------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?

In [72]:
case.where('case_late=="YES"')\
.groupBy('service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



11. Does number of days late depend on department?

In [90]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1318                 
 late_SLA             | 319                  
 case_age_SLA         | 319                  
only showing top 1 row



In [93]:
# YES
(
case.where('case_late=="YES"')
.groupBy('dept_division')
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)
)

+-----------------------------+------------------+
|dept_division                |avg(num_days_late)|
+-----------------------------+------------------+
|Code Enforcement (Internal)  |210.89201994318182|
|311 Call Center              |87.68385942150394 |
|Shops                        |76.87018194568    |
|Code Enforcement (IntExp)    |76.41362019978536 |
|Director's Office Horizontal |76.07892152387127 |
|Engineering Division         |67.31528481452672 |
|Reservations                 |66.03116319       |
|Dangerous Premise            |56.045185022438   |
|Code Enforcement             |48.6743311339552  |
|Tree Crew                    |33.85895311389411 |
|Dangerous Premise (IntExp)   |26.794363426666667|
|Trades                       |24.36677159238625 |
|Graffiti (IntExp)            |24.28088208908296 |
|Field Operations             |23.458633245820124|
|Shops (Internal)             |20.631614157630324|
|Clean and Green Natural Areas|13.450237682142857|
|Solid Waste                  |

12. How do number of days late depend on department and request type?

In [96]:
case.where('case_late=="YES"')\
.groupBy('dept_division', 'service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)


+---------------------------+----------------------------------------+------------------+
|dept_division              |service_request_type                    |avg(num_days_late)|
+---------------------------+----------------------------------------+------------------+
|Code Enforcement (Internal)|Zoning: Recycle Yard                    |210.89201994318182|
|Code Enforcement           |Zoning: Junk Yards                      |200.20517608494276|
|Code Enforcement           |Structure/Housing Maintenance           |190.20707698509807|
|Code Enforcement           |Donation Container Enforcement          |171.09115313942615|
|Code Enforcement (IntExp)  |Storage of Used Mattress                |163.96812829714287|
|Code Enforcement (IntExp)  |Labeling for Used Mattress              |162.43032902285717|
|Code Enforcement (IntExp)  |Record Keeping of Used Mattresses       |153.99724039428568|
|Code Enforcement (IntExp)  |Signage Requied for Sale of Used Mattr  |151.63868055333333|
|Signals  